# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
# Name: Sahar Agranov
# ID: 316263045
# Details Student 2:


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
import string

# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip

Cached file found: C:\Users\agran\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\agran\AppData\Local\Temp\tmp2x289giy\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [5]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\agran\OneDrive - Holon Institute of Technology\2. Year 2\Semester C\Machine Learning\Assignment_3 - Text_Analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)


### Your implementation:
Write your code solution in the following code-cells

# Clean Text Function
### Removing punctuation from the dataframe

In [11]:
def clean_text(text):
    text = text.translate(str.maketrans("","",string.punctuation))
    return text

# Pre process the data

In [12]:
# Preprocess the text and clean it
df_train['cleaned_story'] = df_train['story'].apply(clean_text)

# Separate features (X) and labels (y)
X = df_train['cleaned_story']
y = df_train['gender']

# CountVectorizer

In [13]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3),
                             analyzer='word')

# Fit and transform the text into a BoW representation
X_countvec = vectorizer.fit_transform(X)


# TF-IDF

In [14]:
# Initialize TF_IDFVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))

# Fit and transform
X_tfidf = tfidf_vectorizer.fit_transform(X)

# DecisionTreeClassifier

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_countvec, y, test_size=0.1)

param_grid = {
    'criterion': ['gini'],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

classifier = DecisionTreeClassifier()
grid_search = GridSearchCV(classifier, param_grid,cv= 10, scoring='f1_macro')
grid_search.fit(X_train, y_train)


print("Best Hyperparameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini'], 'max_depth': [5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='f1_macro')

Best Hyperparameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 5}
Best F1 Score: 0.6160136903920089


# Trying with TF/IDF

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.1)

param_grid = {
    'criterion': ['gini'],
    'max_depth': [ 5, 10,],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(classifier, param_grid, scoring='f1_macro')
grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)


GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini'], 'max_depth': [5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='f1_macro')

Best Hyperparameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2}
Best F1 Score: 0.6145162110084934


# Trying SVM

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.1)
svm_classifier = LinearSVC( C=1.0)
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

f1_macro = f1_score(y_test, y_pred, average='macro')
print("F1_macro Score:", f1_macro)

linear_svc_classifier = LinearSVC(class_weight='balanced')

param_grid = {
    'C': [0.1, 1, 10],
    'loss': ['hinge', 'squared_hinge'],
    'max_iter': [1000, 2000]
}

grid_search = GridSearchCV(estimator=linear_svc_classifier, param_grid=param_grid, scoring='f1_macro', cv=10)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Hyperparameters:", best_params)
print("Best F1 Score:", best_score)

best_classifier = grid_search.best_estimator_
y_pred = best_classifier.predict(X_test)


LinearSVC()

F1_macro Score: 0.43703703703703706


GridSearchCV(cv=10, estimator=LinearSVC(class_weight='balanced'),
             param_grid={'C': [0.1, 1, 10], 'loss': ['hinge', 'squared_hinge'],
                         'max_iter': [1000, 2000]},
             scoring='f1_macro')

Best Hyperparameters: {'C': 0.1, 'loss': 'hinge', 'max_iter': 1000}
Best F1 Score: 0.5412976459070782


# Trying KNN


In [20]:
X_train_vec, X_test_vec, y_train_vec, y_test_vec = train_test_split(X_countvec, y, test_size=0.1)
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X_tfidf, y, test_size=0.1)

k = 5

knn_classifier_vec = KNeighborsClassifier(n_neighbors=k)
knn_classifier_vec.fit(X_train_vec, y_train_vec)
y_pred_vec = knn_classifier_vec.predict(X_test_vec)

macro_f1_test_vec = f1_score(y_test_vec, y_pred_vec, average='macro')
print("Macro F1 Score on Test Data - Vec:", macro_f1_test_vec)

knn_classifier_tf = KNeighborsClassifier(n_neighbors=k)
knn_classifier_tf.fit(X_train_tf, y_train_tf)
y_pred_tf = knn_classifier_tf.predict(X_test_tf)

macro_f1_test_tf = f1_score(y_test_tf, y_pred_tf, average='macro')
print("Macro F1 Score on Test Data - TF:", macro_f1_test_tf)

param_grid = {
    'n_neighbors': [2, 3, 5, 7, 10, 12],
    'metric': ['euclidean', 'manhattan'],
}

grid_search_vec = GridSearchCV(estimator=knn_classifier_vec, param_grid=param_grid, scoring='f1_macro', cv=10)
grid_search_tf = GridSearchCV(estimator=knn_classifier_tf, param_grid=param_grid, scoring='f1_macro', cv=10)

grid_search_vec.fit(X_train_vec, y_train_vec)
grid_search_tf.fit(X_train_tf, y_train_tf)

best_params_vec = grid_search_vec.best_params_
best_score_vec = grid_search_vec.best_score_

best_params_tf = grid_search_tf.best_params_
best_score_tf = grid_search_tf.best_score_



KNeighborsClassifier()

Macro F1 Score on Test Data - Vec: 0.4242424242424242


KNeighborsClassifier()

Macro F1 Score on Test Data - TF: 0.6963572267920093


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [2, 3, 5, 7, 10, 12]},
             scoring='f1_macro')

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [2, 3, 5, 7, 10, 12]},
             scoring='f1_macro')

In [21]:
print("Best Hyperparameters:", best_params_vec)
print("Best F1 Score:", best_score_vec)

print("Best Hyperparameters:", best_params_tf)
print("Best F1 Score:", best_score_tf)

Best Hyperparameters: {'metric': 'euclidean', 'n_neighbors': 3}
Best F1 Score: 0.484234740537749
Best Hyperparameters: {'metric': 'euclidean', 'n_neighbors': 10}
Best F1 Score: 0.6290374073557267


# Testing the best model on the Test data

In [22]:
# Preprocess the text and clean it
df_test['cleaned_story'] = df_test['story'].apply(clean_text)

### Using KNN with the best hyperparameters

In [23]:
# Create a KNN classifier with the desired settings
knn_classifier_tf = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_classifier_tf.fit(X_train_tf, y_train_tf)

# Tokenizing
X_new = tfidf_vectorizer.transform(df_test['cleaned_story'])

predicted_gender = knn_classifier_tf.predict(X_new)

# Create a DataFrame with the test example IDs and predicted gender labels
df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'Predicted_Gender': predicted_gender})


KNeighborsClassifier(metric='euclidean', n_neighbors=3)

In [24]:
df_predicted.head()

,test_example_id,Predicted_Gender
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


In [25]:
df_predicted.tail()

,test_example_id,Predicted_Gender
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [26]:
df_predicted.to_csv('classification_results.csv',index=False)